In [6]:
import psycopg
import requests
import json
import time

# -----------------------
# Configuration
# -----------------------
conversation_file_path = "C:\\Users\\lenovo\\Chatbot-RAG\\data\\TRANS_TXT\\017_00000012.txt"
DB_CONNECTION_STR = "dbname=postgres user=postgres password=zaineb host=localhost port=5434"

# OLLAMA
OLLAMA_URL = "http://localhost:11434"
OLLAMA_EMBED_MODEL = "nomic-embed-text"
LLM_MODEL = "gemma2:2b"

VECTOR_DIM = 4096
EMBED_TIMEOUT = 60
GENERATE_TIMEOUT = 90
BATCH_SIZE = 5

# -----------------------
# Fonctions utilitaires
# -----------------------
def detect_encoding(file_path: str) -> str:
    encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                file.read()
            return encoding
        except UnicodeDecodeError:
            continue
    return 'latin-1'

def create_conversation_list(file_path: str) -> list[str]:
    encoding = detect_encoding(file_path)
    print(f"Encodage détecté: {encoding}")
    
    with open(file_path, "r", encoding=encoding) as file:
        text = file.read()
        text_list = text.split("\n")
        filtered_list = [
            chaine.removeprefix("     ")
            for chaine in text_list
            if chaine.strip() and not chaine.startswith("<")
        ]
        print(f"Nombre de phrases: {len(filtered_list)}")
        print("Premières phrases:", filtered_list[:3])
        return filtered_list

def check_ollama_health() -> bool:
    try:
        response = requests.get(f"{OLLAMA_URL}/api/tags", timeout=5)
        return response.status_code == 200
    except:
        return False

def calculate_embeddings_ollama(text: str, retry_count: int = 3) -> list[float]:
    payload = {"model": OLLAMA_EMBED_MODEL, "prompt": text}
    
    for attempt in range(retry_count):
        try:
            response = requests.post(
                f"{OLLAMA_URL}/api/embeddings", 
                json=payload, 
                timeout=EMBED_TIMEOUT
            )
            response.raise_for_status()
            embedding = response.json().get("embedding", [])
            if embedding:
                return embedding
        except requests.exceptions.Timeout:
            print(f"  ⏱️ Timeout tentative {attempt + 1}/{retry_count} pour: {text[:50]}...")
            if attempt < retry_count - 1:
                time.sleep(2)
        except Exception as e:
            print(f"  ❌ Erreur tentative {attempt + 1}/{retry_count}: {e}")
            if attempt < retry_count - 1:
                time.sleep(2)
    
    print(f"  ⚠️ Échec après {retry_count} tentatives pour: {text[:50]}...")
    return []

def embedding_to_pgvector_format(emb: list[float]) -> str:
    return "[" + ",".join(map(str, emb)) + "]"

def save_embedding(corpus: str, embedding: list[float], cursor) -> None:
    emb_literal = embedding_to_pgvector_format(embedding)
    cursor.execute(
        """INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s::vector)""",
        (corpus, emb_literal)
    )

def similar_corpus(input_corpus: str, cursor, top_k: int = 10) -> list[tuple]:
    """Recherche avec distance cosine - TOP_K AUGMENTÉ À 10"""
    emb = calculate_embeddings_ollama(input_corpus)
    if not emb:
        return []
    emb_literal = embedding_to_pgvector_format(emb)
    
    # Distance cosine (<=>)
    cursor.execute(
        f"""
        SELECT id, corpus, embedding <=> %s::vector AS distance
        FROM embeddings
        ORDER BY embedding <=> %s::vector
        LIMIT {top_k}
        """,
        (emb_literal, emb_literal)
    )
    return cursor.fetchall()

def generate_answer_ollama(user_query: str, context_texts: list[str]) -> str:
    if not context_texts:
        return "Aucun contexte trouvé pour répondre à la question."
    
    # Numéroter clairement chaque élément de contexte
    context = "\n".join([f"[{i+1}] {text}" for i, text in enumerate(context_texts)])
    
    # Prompt détaillé avec instructions strictes
    prompt = f"""Tu es un assistant qui répond en utilisant UNIQUEMENT les informations du contexte.

CONTEXTE (conversation entre hotesse 'h:' et client 'c:'):
{context}

QUESTION: {user_query}

INSTRUCTIONS IMPORTANTES:
1. Lis TOUT le contexte attentivement ligne par ligne
2. Liste TOUS les organismes, lieux et informations mentionnés
3. N'oublie AUCUN détail (noms d'organismes, adresses, dates)
4. Structure ta réponse clairement
5. N'invente RIEN qui n'est pas dans le contexte

RÉPONSE COMPLÈTE ET DÉTAILLÉE:"""
    
    payload = {
        "model": LLM_MODEL,
        "prompt": prompt,
        "temperature": 0.05,  # Très déterministe
        "stream": True,
        "options": {
            "num_predict": 250,  # Plus de tokens pour réponse complète
            "top_k": 5,
            "top_p": 0.3
        }
    }
    
    try:
        response = requests.post(
            f"{OLLAMA_URL}/api/generate", 
            json=payload, 
            stream=True,
            timeout=GENERATE_TIMEOUT
        )
        response.raise_for_status()
        
        full_response = ""
        print("   ", end="", flush=True)
        for line in response.iter_lines():
            if line:
                try:
                    json_response = json.loads(line)
                    chunk = json_response.get("response", "")
                    full_response += chunk
                    print(chunk, end="", flush=True)
                    if json_response.get("done", False):
                        break
                except json.JSONDecodeError:
                    continue
        
        print()
        return full_response.strip() if full_response else "Erreur : réponse vide"
    
    except requests.exceptions.Timeout:
        return "⏱️ Timeout : essayez une question plus simple."
    except Exception as e:
        return f"Erreur : {e}"

# -----------------------
# Pipeline principal
# -----------------------
def main():
    print("=" * 60)
    print(f"SYSTÈME RAG avec OLLAMA ({LLM_MODEL})")
    print("=" * 60)
    
    print("\n🔍 Vérification d'Ollama...")
    if not check_ollama_health():
        print("❌ Ollama n'est pas accessible")
        print("   Démarrez-le avec: ollama serve")
        return
    print("✅ Ollama est accessible")
    
    corpus_list = create_conversation_list(conversation_file_path)
    if not corpus_list:
        print("❌ Aucun texte trouvé!")
        return
    
    print("\n🗄️  Connexion à PostgreSQL...")
    try:
        with psycopg.connect(DB_CONNECTION_STR) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                print("\n🔧 Préparation de la base...")
                cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
                cur.execute(f"""
                    CREATE TABLE IF NOT EXISTS embeddings (
                        id SERIAL PRIMARY KEY,
                        corpus TEXT,
                        embedding vector({VECTOR_DIM})
                    )
                """)
                cur.execute("DELETE FROM embeddings")
                
                print(f"\n🤖 Génération des embeddings ({len(corpus_list)} phrases)...")
                print("   Patience...")
                
                successful = 0
                failed = 0
                
                for i, corpus in enumerate(corpus_list, 1):
                    if corpus.strip():
                        print(f"   [{i}/{len(corpus_list)}] ", end="", flush=True)
                        emb = calculate_embeddings_ollama(corpus)
                        if emb:
                            save_embedding(corpus, emb, cur)
                            successful += 1
                            print("✓")
                        else:
                            failed += 1
                            print("✗")
                        
                        if i % BATCH_SIZE == 0:
                            time.sleep(0.5)
                
                conn.commit()
                print(f"\n✅ Embeddings: {successful} réussis, {failed} échoués")
                
                if successful == 0:
                    print("❌ Impossible de continuer sans embeddings")
                    return
                
                print("\n" + "="*60)
                print("💬 MODE INTERACTIF")
                print("="*60)
                print("Tapez 'quit' pour sortir\n")
                
                while True:
                    user_query = input("❓ Question : ").strip()
                    if user_query.lower() in ['quit', 'exit', 'q']:
                        print("👋 Au revoir!")
                        break
                    if not user_query:
                        continue
                    
                    print("\n🔍 Recherche (top-10)...")
                    results = similar_corpus(user_query, cur, top_k=10)
                    
                    if results:
                        context_texts = [r[1] for r in results]
                        print("\n📄 Contexte trouvé :")
                        for i, (_, corpus, distance) in enumerate(results):
                            print(f"  [{i+1}] {corpus[:70]}... (dist: {distance:.3f})")
                        
                        print("\n🤖 Génération de la réponse...\n")
                        answer = generate_answer_ollama(user_query, context_texts)
                        print(f"\n{'='*60}")
                        print(f"💡 RÉPONSE:\n{answer}")
                        print(f"{'='*60}\n")
                    else:
                        print("❌ Aucun contexte trouvé.\n")
    
    except Exception as e:
        print(f"❌ Erreur : {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

SYSTÈME RAG avec OLLAMA (gemma2:2b)

🔍 Vérification d'Ollama...
✅ Ollama est accessible
Encodage détecté: latin-1
Nombre de phrases: 43
Premières phrases: ['h: U B S bonjour', "c: oui bonjour e j'appelle je sais pas si j'appelle au bon endroit e", 'h: je vous écoute']

🗄️  Connexion à PostgreSQL...

🔧 Préparation de la base...

🤖 Génération des embeddings (43 phrases)...
   Patience...
   [1/43] ✓
   [2/43] ✓
   [3/43] ✓
   [4/43] ✓
   [5/43] ✓
   [6/43] ✓
   [7/43] ✓
   [8/43] ✓
   [9/43] ✓
   [10/43] ✓
   [11/43] ✓
   [12/43] ✓
   [13/43] ✓
   [14/43] ✓
   [15/43] ✓
   [16/43] ✓
   [17/43] ✓
   [18/43] ✓
   [19/43] ✓
   [20/43] ✓
   [21/43] ✓
   [22/43] ✓
   [23/43] ✓
   [24/43] ✓
   [25/43] ✓
   [26/43] ✓
   [27/43] ✓
   [28/43] ✓
   [29/43] ✓
   [30/43] ✓
   [31/43] ✓
   [32/43] ✓
   [33/43] ✓
   [34/43] ✓
   [35/43] ✓
   [36/43] ✓
   [37/43] ✓
   [38/43] ✓
   [39/43] ✓
   [40/43] ✓
   [41/43] ✓
   [42/43] ✓
   [43/43] ✓

✅ Embeddings: 43 réussis, 0 échoués

💬 MODE INTERACTIF
Tapez